In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
df1 = pd.read_excel('Finance_Pull_Dept_level_Feb (Inc Hist).xlsx')

In [4]:
mth = pd.DataFrame({'desc': ['Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec', 'Jan'], 
                    'nbr': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]}) 
df1['desc'] = df1['Unnamed: 1'].str[:3] 
df1 = df1.merge(mth, on='desc', how='left') 
df1 = df1[df1['Unnamed: 4'].notnull()] 
df1['acctg_dept_nbr'] = df1['Unnamed: 3'].str.extract('(\d+)') 
df1 = df1[df1['acctg_dept_nbr'].notnull()]

<>:6: SyntaxWarning: invalid escape sequence '\d'
<>:6: SyntaxWarning: invalid escape sequence '\d'
/var/folders/xk/pl3jmg716fl_105yxbpsvf7w0000gp/T/ipykernel_55379/1544749903.py:6: SyntaxWarning: invalid escape sequence '\d'
  df1['acctg_dept_nbr'] = df1['Unnamed: 3'].str.extract('(\d+)')


In [7]:
df1.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,desc,nbr,acctg_dept_nbr
8,FYE19,Feb-454,YearTotal,Wd.010 AUTOMOTIVE,471396938.79,0,0,0,0,0,0,0,NaN,Feb,1.0,010
9,FYE19,Feb-454,YearTotal,Wd.037 AUTO SERVICE,42151656.76,0,0,0,0,0,0,0,NaN,Feb,1.0,037
10,FYE19,Feb-454,YearTotal,Wd.009 SPORTS AND OUTDOORS,252323188.05,0,0,0,0,0,0,0,NaN,Feb,1.0,009
11,FYE19,Feb-454,YearTotal,Wd.011 HARDWARE,259612260.96,0,0,0,0,0,0,0,NaN,Feb,1.0,011
12,FYE19,Feb-454,YearTotal,Wd.012 DO IT YOURSELF,74980500.49,0,0,0,0,0,0,0,NaN,Feb,1.0,012


In [9]:
df2 = pd.DataFrame() 
df2['wm_full_yr_nbr'] =("20" + df1['Unnamed: 0'].str[-2:]).astype(int) - 1 
df2['wm_mth_nbr'] = df1['nbr'] 
df2['acctg_dept_nbr'] = df1['acctg_dept_nbr'].astype(int) 
df2['BIS'] = df1['Unnamed: 4'] - df1['Unnamed: 7'] - df1['Unnamed: 8'] - df1['Unnamed: 11'] 
df2['CORE'] = df1['Unnamed: 5'] 
df2['MP'] = df1['Unnamed: 6'] 
df2['SFS'] = df1['Unnamed: 8'] 
df2['PUT'] = df1['Unnamed: 7'] 
df2['Scheduled Pickup'] = df1['Unnamed: 9'] 
df2['Scheduled Delivery'] = df1['Unnamed: 10'] 


In [11]:
df3 = pd.melt(df2, id_vars=['wm_full_yr_nbr', 'wm_mth_nbr', 'acctg_dept_nbr'], value_vars=['BIS', 'CORE', 'MP','PUT','SFS','Scheduled Pickup','Scheduled Delivery'])
df3 = df3.rename(columns={'variable':'channel_detail', 'value':'wm_sales'})

In [13]:
def assign_channel(channel_detail):
    if channel_detail in ['CORE', 'MP','PUT','SFS','Scheduled Pickup','Scheduled Delivery']:
        return 'Online'
    elif channel_detail == 'BIS':
        return 'BIS'
    else:
        return 'Other'

df3['channel'] = df3['channel_detail'].apply(assign_channel)

In [15]:
df3.to_csv('Merch_Planner_FY25Jan.csv', index=False)